In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(':memory:')

In [3]:
table_names = [
    'Categories',   # категории товаров
    'Customers',    # заказчики
    'Employees',    # сотрудники
    'OrderDetails', # детали заказов
    'Orders',       # заказы
    'Products',     # товары
    'Shippers',     # перевозчики
    'Suppliers',    # поставщики
]

In [4]:
for table in table_names:
    data = pd.read_csv(f'{table}.csv', sep = ';')
    data.to_sql(table, con, index=False, if_exists='replace')

In [5]:
for i in table_names:
    display(i, pd.read_sql(
        f'''
        SELECT * FROM {i}
        ''',
        con,
    ).head(1))


'Categories'

,CategoryID,CategoryName,Description
0,1,Beverages,"Soft drinks, coffees, teas, beers, and ales"


'Customers'

,CustomerID,CustomerName,ContactName,Address,City,PostalCode,Country
0,1,Alfreds Futterkiste,Maria Anders,Obere Str. 57,Berlin,12209,Germany


'Employees'

...

,ProductID,ProductName,SupplierID,CategoryID,Unit,Price
0,1,Chais,1,1,10 boxes x 20 bags,18.0


'Shippers'

,ShipperID,ShipperName,Phone
0,1,Speedy Express,(503) 555-9831


'Suppliers'

,SupplierID,SupplierName,ContactName,Address,City,PostalCode,Country,Phone
0,1,Exotic Liquid,Charlotte Cooper,49 Gilbert St.,Londona,EC1 4SD,UK,(171) 555-2222


### Все задания нужно выполнить только с помощью sql

1. Выберите заказчиков из Германии, Франции и Мадрида, выведите их название, страну и адрес.

In [6]:
pd.read_sql(
    '''
    SELECT CustomerName	, Country, Address
    FROM Customers c
    Where c.Country in ('Germany', 'France') or City = 'Madrid'
     
    ''',
    con,
)

,CustomerName,Country,Address
0,Alfreds Futterkiste,Germany,Obere Str. 57
1,Blauer See Delikatessen,Germany,Forsterstr. 57
2,Blondel père et fils,France,"24, place Kléber"
3,Bólido Comidas preparadas,Spain,"C/ Araquil, 67"
4,Bon app',France,"12, rue des Bouchers"
5,Drachenblut Delikatessend,Germany,Walserweg 21
6,Du monde entier,France,"67, rue des Cinquante Otages"
7,FISSA Fabrica Inter. Salchichas S.A.,Spain,"C/ Moralzarzal, 86"
8,Folies gourmandes,France,"184, chaussée de Tournai"
9,Frankenversand,Germany,Berliner Platz 43


2. Выберите топ 3 страны по количеству заказчиков, выведите их названия и количестnunique записей.

In [7]:
pd.read_sql(
    '''
    SELECT distinct Country, count(*) count_ord FROM Customers
    group by Country
    order by 2 desc
    limit 3
    ''',
    con,
)

,Country,count_ord
0,USA,13
1,Germany,11
2,France,11


3. Выберите перевозчика, который отправил 10-й по времени заказ, выведите его название, и дату отправления.

In [8]:
display(pd.read_sql(
    '''
    select ShipperName, OrderDate from (
        SELECT *, row_number() over() numb FROM Orders o left join Shippers s on o.ShipperID = s.ShipperID
        order by substr(OrderDate, 7,4), substr(OrderDate, 4,2), substr(OrderDate, 1,2)
        )
    where numb = 10
    ''',
    con, parse_dates={'OrderDate':{'format':'%d.%m.%Y'}}
        )
     )
display(pd.read_sql(
    '''
        SELECT s.ShipperName, o.OrderDate FROM Orders o left join Shippers s on o.ShipperID = s.ShipperID
        order by substr(OrderDate, 7,4), substr(OrderDate, 4,2), substr(OrderDate, 1,2)
        limit 1 offset 9
    ''',
    con, parse_dates={'OrderDate':{'format':'%d.%m.%Y'}}
                    )
    )

,ShipperName,OrderDate
0,Federal Shipping,1996-07-16


,ShipperName,OrderDate
0,Federal Shipping,1996-07-16


4. Выберите самый дорогой заказ, выведите список товаров с их ценами.

In [9]:
pd.read_sql(
    '''
    SELECT o.OrderID,ProductName,Price, Quantity   FROM Orders o left join OrderDetails od on o.OrderID = od.OrderID
    left join Products p on od.ProductID = p.ProductID
    where o.OrderID in (
    
    
    SELECT o.OrderID  FROM Orders o left join OrderDetails od on o.OrderID = od.OrderID
    left join Products p on od.ProductID = p.ProductID
    group by o.OrderID 
    order by sum(Quantity*Price) desc
    limit 1)
    ''',
    con,
)

,OrderID,ProductName,Price,Quantity
0,10372,Sir Rodney's Marmalade,81.0,12
1,10372,Côte de Blaye,263.5,40
2,10372,Camembert Pierrot,34.0,70
3,10372,Mozzarella di Giovanni,34.8,42


5. Какой товар больше всего заказывали по количеству единиц товара, выведите его название и количество единиц в каждом из заказов.

In [10]:
pd.read_sql(
    '''  
    select ProductName ,OrderID, Quantity  FROM OrderDetails o left join Products p on o.ProductID = p.ProductID
    where o.ProductID in (
    
    SELECT p.ProductID FROM Orders o left join OrderDetails od on o.OrderID = od.OrderID
    left join Products p on od.ProductID = p.ProductID
    group by p.ProductID
    order by sum(Quantity) desc
    limit 1)
    order by 2 desc
    ''',
    con,
)

,ProductName,OrderID,Quantity
0,Gorgonzola Telino,10423,14
1,Gorgonzola Telino,10393,32
2,Gorgonzola Telino,10390,60
3,Gorgonzola Telino,10376,42
4,Gorgonzola Telino,10374,30
5,Gorgonzola Telino,10363,20
6,Gorgonzola Telino,10359,70
7,Gorgonzola Telino,10356,30
8,Gorgonzola Telino,10342,56
9,Gorgonzola Telino,10335,25


6. Выведите топ 5 поставщиков по количеству заказов, выведите их названия, страну, контактное лицо и телефон.

In [11]:
pd.read_sql(
    '''  
    SELECT SupplierName,Country	, ContactName,Phone   FROM OrderDetails od left join Products p on od.ProductID = p.ProductID
    left join Suppliers s on s.SupplierID = p. SupplierID
    group by p.SupplierID
    order by sum(Quantity) desc
    limit 5
    ''',
    con,
)

,SupplierName,Country,ContactName,Phone
0,"Pavlova, Ltd.",Australia,Ian Devling,(03) 444-2343
1,Norske Meierier,Norway,Beate Vileid,(0)2-953010
2,Formaggi Fortini s.r.l.,Italy,Elio Rossi,(0544) 60323
3,Gai pâturage,France,Eliane Noz,38.76.98.06
4,Plutzer Lebensmittelgroßmärkte AG,Germany,Martin Bein,(069) 992755


7. Какую категорию товаров заказывали больше всего по стоимости в Бразилии, выведите страну, название категории и сумму.

In [12]:
pd.read_sql(
    ''' 
    SELECT country,CategoryName, sum(Quantity*Price) summ
    FROM Suppliers s left join Products p on p.SupplierID = s.SupplierID 
    left join OrderDetails o on o.ProductID = p.ProductID left join Categories c on p.CategoryID = c.CategoryID
    where country = 'Brazil'
    group by country ,CategoryName
    ''',
    con
)

,Country,CategoryName,summ
0,Brazil,Beverages,711.0


8. Какая разница в стоимости между самым дорогим и самым дешевым заказом из США.

In [13]:
pd.read_sql(
    '''  
    select country, max(summ) max, min(summ) min, max(summ) - min(summ) diff from (
    SELECT country, OrderID, sum(Quantity* Price)  summ
    FROM OrderDetails od left join Products p on od.ProductID = p.ProductID left join Suppliers s on s.SupplierID = p. SupplierID  
    where Country = 'USA'
    group by country, OrderID
    order by 2)
    ''',
    con,
)

,country,max,min,diff
0,USA,3570.0,36.8,3533.2


9. Выведите количество заказов у каждого их трех самых молодых сотрудников, а также имя и фамилию во второй колонке.

In [14]:
pd.read_sql(
    ''' 
    select e.FirstName||' '||LastName Name, count() count_ord FROM Orders o left join Employees e on e.EmployeeID = o. EmployeeID  
    where o.EmployeeID in (
    SELECT EmployeeID
    FROM Employees e
    order by date(substr(BirthDate,7)||'-'||substr(BirthDate,4,2)||'-'||substr(BirthDate,1,2)) desc
    limit 3)
    group by e.EmployeeID
    ''',
    con
)

,Name,count_ord
0,Nancy Davolio,29
1,Janet Leverling,31
2,Anne Dodsworth,6


10. Сколько банок крабового мяса всего было заказано.

In [17]:
pd.read_sql(
    
    ''' 
    SELECT ProductName, Unit, sum(Quantity*substr(Unit,1,2)) tins
    FROM Products p left join OrderDetails od on p.ProductID = od.ProductID
    where ProductName like '%crab%'
    group by ProductName, Unit
    ''',
    con
)

,ProductName,Unit,tins
0,Boston Crab Meat,24 - 4 oz tins,6144
